In [2]:
from catboost import CatBoostClassifier, Pool
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
import copy 

### Перейдем непосредственно к классификации.   
#### Основным нашим методом будет градиентный бустинг от CatBoost.

In [99]:
N_COMPONENTS = 128
class_names = ['Female', 'Male']

Загрузим подготовленный датасет

In [3]:
X_train = np.load('./data/features/train_x.npy')
y_train = np.load('./data/features/train_y.npy')
X_val = np.load('./data/features/val_x.npy')
y_val = np.load('./data/features/val_y.npy')
X_test = np.load('./data/features/test_x.npy')
y_test = np.load('./data/features/test_y.npy')

Так как многие значения векторов X имеют нулевые значения, их размер избыточен, поэтому для начала применим метод главных компонент для понижения размерности

In [30]:
pca = PCA(n_components=N_COMPONENTS)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)

In [31]:
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

В качестве классификатора будет использоваться CatBoostClassifier

In [32]:
train_dataset = Pool(data=X_train_pca, 
                     label=y_train,
                    )
val_dataset = Pool(data=X_val_pca,
                    label=y_val,
                    )
test_dataset = Pool(data=X_test_pca,
                    label=y_test)

In [93]:
clf = CatBoostClassifier(iterations = None,
                        learning_rate=0.01,
                        loss_function='Logloss',
                        verbose=False)

In [94]:
clf.fit(train_dataset)

In [95]:
prediction_val = clf.predict(val_dataset)

In [96]:
print("Accuracy:", np.mean(np.array(prediction_val).flatten() == y_val))

Accuracy: 0.9617930204572804


In [97]:
prediction_test = clf.predict(test_dataset)
print("Accuracy:", np.mean(np.array(prediction_test).flatten() == y_test))

Accuracy: 0.9756047136654952


In [98]:
clf.save_model('./parameters/clf_model.pkl')

Посмотрим на другие метрики классификации

In [100]:
print(classification_report(y_test, prediction_test, target_names=class_names))

              precision    recall  f1-score   support

      Female       0.97      0.99      0.98      2930
        Male       0.98      0.96      0.97      1907

    accuracy                           0.98      4837
   macro avg       0.98      0.97      0.97      4837
weighted avg       0.98      0.98      0.98      4837



In [102]:
import pickle
with open('./parameters/pca.pkl', 'wb') as pickle_file:
    pickle.dump(pca, pickle_file)